In [ ]:
import sys
!{sys.executable} -m pip install pycbc ligo-common --no-cache-dir

     |████████████████████████████████| 6.7 MB 3.9 MB/s 
     |████████████████████████████████| 75 kB 48.1 MB/s 
     |████████████████████████████████| 201 kB 62.9 MB/s 
     |████████████████████████████████| 51 kB 51.6 MB/s 
     |████████████████████████████████| 1.6 MB 32.1 MB/s 
     |████████████████████████████████| 27.4 MB 1.2 MB/s 
     |████████████████████████████████| 55 kB 47.3 MB/s 
     |████████████████████████████████| 3.6 MB 51.2 MB/s 
  Created wheel for lscsoft-glue: filename=lscsoft_glue-2.0.0-cp37-cp37m-linux_x86_64.whl size=416795 sha256=d62036f26385f993236281530003c8cc9deb3d86b4bbc49d44b39320053805d3
  Stored in directory: /tmp/pip-ephem-wheel-cache-hzfky4wy/wheels/9d/ed/9e/1f9cad5e15359dc17954b893dd486760b5cb1ffd97a8405d1e
  Created wheel for ligo-segments: filename=ligo_segments-1.4.0-cp37-cp37m-linux_x86_64.whl size=86649 sha256=2bf3c0c9b42bb92d53eba198614a5115e5fe3143cec9da3c47d737255256d898
  Stored in directory: /tmp/pip-ephem-wheel-cache-hzfky4wy/wheels

In [ ]:
import numpy as np
import math
import pylab
import matplotlib.pyplot as plt
import random
import pycbc
from pycbc.psd import welch, interpolate
from pycbc.psd import interpolate, inverse_spectrum_truncation
from pycbc.noise.gaussian import noise_from_psd
from pycbc.noise.gaussian import frequency_noise_from_psd
from pycbc.filter import matched_filter



N=2048*16  #N is number of samples, N=length/delta_t
fs=2048 #fs is sampling frequnecy
length=16 #duration of segment
delta_f=1.0/16
f_samples = 2048*16 +1
f_lower=30
delta_t=1.0/2048

In [ ]:
from pycbc.psd.analytical import AdVDesignSensitivityP1200087

def get_psd(f_samples, delta_f, low_freq_cutoff):
    psd=AdVDesignSensitivityP1200087(f_samples, delta_f, low_freq_cutoff)
    return psd

from pycbc.noise.gaussian import frequency_noise_from_psd

def get_noise(psd, seed=None):
    noise=frequency_noise_from_psd(psd, seed=seed)
    noise_time = noise.to_timeseries()
    return noise_time

In [ ]:
from pycbc.psd import welch, interpolate

def get_whiten(signal_plus_noise):
    signal_freq_series=signal_plus_noise.to_frequencyseries()
    numerator = signal_freq_series
    psd_to_whiten = interpolate(welch(signal_plus_noise), 1.0 / signal_plus_noise.duration)
    denominator=np.sqrt(psd_to_whiten)
    whiten_freq = (numerator / denominator)
    whiten=whiten_freq.to_timeseries().highpass_fir(30., 512).lowpass_fir(300.0, 512)
    return whiten

def get_8s(whiten, signal_peak_index=None):
    whiten.start_time = 0
    cropped = whiten.time_slice(0,8)
    return cropped

psd=get_psd(f_samples, delta_f, f_lower)


In [ ]:
def NOISE_GENERATOR(samples):
  for i in range(samples):
    seed =  random.randint(1, 1000)
    noise=get_noise(psd, seed)
    whiten = get_whiten (noise)
    noise_signal = get_8s(whiten)
    # pylab.plot(noise_signal.sample_times, noise_signal)
    # print(noise_signal[0])
    my_dir = '/content/Negative Test DATA/' # write the file name in which you need to put the data
    name = 101000000+i+1
    np.save(my_dir + str(name), noise_signal)


In [ ]:
NOISE_GENERATOR(4096)

In [ ]:
import shutil
shutil.make_archive('/content/Negative_Test_DATA', 'zip', 'Negative Test DATA')

'/content/Negative_Test_DATA.zip'

In [ ]:
# ! unzip -q /content/Negative_Training_DATA.zip

In [ ]:
# from google.colab import files
# files.download('/content/Negative_Training_DATA.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)


Mounted at /content/gdrive


In [ ]:
!cp Negative_Test_DATA.zip '/content/gdrive/My Drive/'
!ls -lt '/content/gdrive/My Drive/' 

total 2966806
-rw------- 1 root root 1033877195 Dec  4 08:00  Negative_Test_DATA.zip
-rw------- 1 root root  258469289 Dec  4 07:54  Negative_Validation_DATA.zip
-rw------- 1 root root 1733558272 Dec  4 07:33  Negative_Training_DATA.zip
-rw------- 1 root root        151 Nov 29 06:03 'Internship PPT.gslides'
-rw------- 1 root root    7275567 Oct  2 19:09  submission_2.webm
-rw------- 1 root root     801289 Oct  2 12:42  submission1.webm
drwx------ 2 root root       4096 Oct  2 12:27  submission_1
-rw------- 1 root root        151 Sep 26 07:39 'Masters Info.gslides'
-rw------- 1 root root    1274470 Aug 17 17:28  Mithil_TeamA_Avionics.pdf
-rw------- 1 root root    1274470 Aug 17 17:26  HPR_Recovery_System.pdf
-rw------- 1 root root        151 Aug 17 07:32 'TPO TY 2018-22 EX.gsheet'
-rw------- 1 root root    1461625 Aug 15 17:26 'Mithil_TeamA_Avionics (1).pdf'
-rw------- 1 root root        151 Aug 10 07:34 'our product.pptm.gslides'
drwx------ 2 root root       4096 Jun 29 15:25 'Colab No